In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1> Goal</h1>

<h4>The goal of this notebook is to simulate production-grade sql-querying on the Lending club dataset by performing initial EDA. In this initial phase we will be checking/cleaning for null values, negative values , duplicates and outliers and erroraneous categorical values for both the Accepted and Rejected loans dataset. From here, we will perfomr deep EDA via python in the next notebook. </h4>

In [2]:
import pandas as pd
import polars as pl
import sqlite3

<h4> Let's  create/connect to a database that we will use to simulate  SQL querying.</h4>

In [3]:
Connection = sqlite3.connect('Lending_Club_Loans.db')

<h4> The following are suggested columns for modeling based on Chatgpt, the original data dictionary was not present on the Lending Data Club website. I asked chatgpt for a description of all the columns and then searched online to see if i could find the descriptions for each column to verifiy correctness.</h4>

In [56]:
modeling_columns = [
    'loan_amnt',
    'funded_amnt',
    'funded_amnt_inv',
    'term',
    'int_rate',
    'installment',
    'grade',
    'sub_grade',
    'emp_length',
    'home_ownership',
    'annual_inc',
    'verification_status',
    'purpose',
    'addr_state',
    'dti',
    'delinq_2yrs',
    'fico_range_low',
    'fico_range_high',
    'inq_last_6mths',
    'open_acc',
    'pub_rec',
    'revol_bal',
    'revol_util',
    'total_acc',
    'collections_12_mths_ex_med',
    'application_type',
    'acc_now_delinq',
    'tot_coll_amt',
    'tot_cur_bal',
    'open_acc_6m',
    'open_il_12m',
    'open_il_24m',
    'total_bal_il',
    'open_rv_12m',
    'open_rv_24m',
    'max_bal_bc',
    'all_util',
    'total_rev_hi_lim',
    'inq_fi',
    'total_cu_tl',
    'inq_last_12m',
    'loan_status',
    'issue_d'
]

<h4> I examined years  2007-2018 in increments to avoid loading the entire dataset once and found that there is only around 40 transactions that resulted in a default. In order to have concrete data for modeling purposes , we either have to use some form of SMOTE or revisit what we consider 'Defaulting" on payments. </h4>

<h4> We will examine years 2007-2010 as to avoid using the entire dataset, and this will give us an ample of new test data to work when we deploy our models.</h4>

In [57]:
# We will examine the datasets separtely for intial findings via SQL
# We will examine years 2007-2010
Accepted_Loans = (pl.read_csv("/kaggle/input/lending-club/accepted_2007_to_2018Q4.csv.gz",skip_rows_after_header=1 ,columns=modeling_columns).filter(pl.col("issue_d").str.contains(r"2007|2008|2009|2010") ) ) # This will gather only data from 2007-2010

In [59]:

Rejected_Loans = pl.read_csv("/kaggle/input/lending-club/rejected_2007_to_2018Q4.csv.gz").filter(pl.col('Application Date').str.contains(r"2007|2008|2009|2010")) # This will gather only data from 2007-2010

In [60]:
# polars doesnt have built in sql querying, we have to convert back into pandas and then transfer dataframe to sql database as a table
Accepted_Loans.to_pandas().to_sql('Accepted',Connection,if_exists='replace',index=False) 
Rejected_Loans.to_pandas().to_sql('Rejected',Connection,if_exists='replace',index=False)

200422

<h2> Data preprocessing </h2>

<h3>Checking for Nulls, Duplicates, Negative values, outliers and Categorical errors for Accepted Loans.</h3> 

In [62]:
# Lets compare the number of records in the Accepted and Rejected Loans
Query = " Select 'Accepted' as TableName ,count(*) as Records from Accepted UNION ALL Select 'Rejected' as TableName , count(*) as Records from Rejected " # Let's check the number of records for each table for quick comparison
Result = pd.read_sql(Query,Connection)
Result

,TableName,Records
0,Accepted,20814
1,Rejected,200422


<h4>There's around 10x more rejected loans than accepted from 2007-2010. From this , we can assume that the Lending Club has strict rules in terms of what qualifies as an acceptable request for a loan.</h4>

<h3> Accepted Loans Negative Value Count for numeric columns.</h3>

In [63]:
# Lets check for negaive values, first we will gather all numeric columns
Query = " SELECT name from pragma_table_info('Accepted') WHERE type IN ('INTEGER', 'REAL', 'NUMERIC', 'DECIMAL', 'FLOAT', 'DOUBLE')" # let's check for numeric columns
Numeric_Cols = pd.read_sql(Query,Connection)
Numeric_Cols

,name
0,loan_amnt
1,funded_amnt
2,funded_amnt_inv
3,int_rate
4,installment
5,annual_inc
6,dti
7,delinq_2yrs
8,fico_range_low
9,fico_range_high


In [64]:
Numeric_Cols = list(Numeric_Cols['name']) # Convert numeric_cols dataframe into a list
print(Numeric_Cols)

['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_12m', 'open_il_24m', 'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m']


In [67]:
Cols_Neg_Count = pd.DataFrame({ 'Column':[],'Negative_Count':[]}) # create an empty data frame to store the negaive count for each column into


for col in Numeric_Cols:
    Query = f" Select count(*) as Negative_Count from Accepted WHERE {col} < 0 " # check for negative values for the given column 'col'
    Neg_Count_Result = pd.read_sql(Query,Connection) # store dataframe result into a variable
    Cols_Neg_Count = pd.concat([ Cols_Neg_Count, pd.DataFrame({ 'Column':[col],'Negative_Count':[ Neg_Count_Result['Negative_Count'].iloc[0]]})], ignore_index=True) # merge with empty dataframe

In [68]:
Cols_Neg_Count

,Column,Negative_Count
0,loan_amnt,0.0
1,funded_amnt,0.0
2,funded_amnt_inv,0.0
3,int_rate,0.0
4,installment,0.0
5,annual_inc,0.0
6,dti,0.0
7,delinq_2yrs,0.0
8,fico_range_low,0.0
9,fico_range_high,0.0


<h3> Accepted Loans Null Value Count for numeric columns.</h3>

In [69]:
Accepted_Records = pd.DataFrame({ 'TableName':[],'Null_Percentage':[]}) # lets make an empty dataframe that we will populate with our gathered data from our queries
Total_Rows_Accepted = 20814 # number of records in the Accepted Loans

for col in modeling_columns:
    Query = f" Select {col} as TableName, count(*) as Null_Count from Accepted  WHERE {col}  IS NULL " # check for nulls in each column
    Accepted_Loans_Result = pd.read_sql(Query,Connection) # stores query result in Accepted_Loans_Result Variable
    Null_Count = Accepted_Loans_Result['Null_Count'].iloc[0] # get the total number of null values for that column or attribute 
    Null_Percentage = (Null_Count/Total_Rows_Accepted)*100  # caluclate what percentage of the values for that column are null vlaues
    Accepted_Records = pd.concat([Accepted_Records, pd.DataFrame({ 'TableName': [col],'Null_Percentage': [Null_Percentage]})], ignore_index=True) # add column data to Accepted_Records dataframe, kinda like x = x+1

In [70]:
print(Accepted_Records)

                     TableName  Null_Percentage
0                    loan_amnt         0.000000
1                  funded_amnt         0.000000
2              funded_amnt_inv         0.000000
3                         term         0.000000
4                     int_rate         0.000000
5                  installment         0.000000
6                        grade         0.000000
7                    sub_grade         0.000000
8                   emp_length         1.695974
9               home_ownership         0.000000
10                  annual_inc         0.019218
11         verification_status         0.000000
12                     purpose         0.000000
13                  addr_state         0.000000
14                         dti         0.000000
15                 delinq_2yrs         0.139329
16              fico_range_low         0.000000
17             fico_range_high         0.000000
18              inq_last_6mths         0.139329
19                    open_acc         0

<h4> Based on these results, I'm going to remove columns 27-40 as 100% of the values are missing and thus will not contribute anything to the modeling process. The remaining columns can be imputed with their respective mean values as they are missing <=2% of data. Columns 27-40 were most likely not tracked during 2007-2010, which would explain why they are missing entirely.  </h4>

<h3> Accepted loans check for outliers</h3>

<h4> I'm going to check for outliers in the numeric columns before imputing the data</h4>

In [71]:
# Columns to be imputed with respective mean or median value in accepted loans
Accepted_Columns_Impute = ['annual_inc', 'delinq_2yrs','inq_last_6mths','open_acc','pub_rec', 'revol_util','total_acc','collections_12_mths_ex_med','acc_now_delinq']

# Columns to drop in accepted loans table, may be useful to perform A/B testing  with these features in the future
Accepted_Columns_Drop = ['tot_coll_amt','tot_cur_bal','open_acc_6m','open_il_12m','open_il_24m','total_bal_il','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m']

In [72]:
Box_Whiskers_Accepted_Loans = pd.DataFrame(columns=['TableName', 'q1', 'Median', 'q3', 'lower_bound', 'upper_bound'])

for col in Accepted_Columns_Impute:
    # Get total number of non-null rows for the column
    count_query = f"SELECT COUNT({col}) AS total FROM Accepted WHERE {col} IS NOT NULL"
    total_result = pd.read_sql(count_query, Connection)
    total = total_result['total'].iloc[0]

    # Calculate the offsets for Q1, Median, Q3
    q1_offset = int(total * 0.25)
    median_offset = int(total * 0.5)
    q3_offset = int(total * 0.75)

  # first we will order the data data gathered from the columns to be imputed
  # Then we will calcuate the respective quartiles for each column as q1,median and q3
  # from here we will perform the necessary calculations to find lower and upper bounds for each column
  # and store the result in our empty dataframe for further analysis
    Query = f"""
    WITH ordered AS (
      SELECT {col}
      FROM Accepted
      WHERE {col} IS NOT NULL
      ORDER BY {col}
    )
    SELECT 
      (SELECT {col} FROM ordered LIMIT 1 OFFSET {q1_offset}) AS q1, 
      (SELECT {col} FROM ordered LIMIT 1 OFFSET {median_offset}) AS median,
      (SELECT {col} FROM ordered LIMIT 1 OFFSET {q3_offset}) AS q3
    """
    
    Box_Result = pd.read_sql(Query, Connection) 

    q1 = Box_Result['q1'].iloc[0]
    median = Box_Result['median'].iloc[0]
    q3 = Box_Result['q3'].iloc[0]
    lower_bound = q1 - 1.5 * (q3 - q1)
    upper_bound = q3 + 1.5 * (q3 - q1)

    Box_Whiskers_Accepted_Loans = pd.concat([
        Box_Whiskers_Accepted_Loans,
        pd.DataFrame({'TableName': [col],'q1': [q1],'Median': [median],'q3': [q3],'lower_bound': [lower_bound],'upper_bound': [upper_bound]})], ignore_index=True)


/tmp/ipykernel_35/3901675854.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Box_Whiskers_Accepted_Loans = pd.concat([


In [73]:
Box_Whiskers_Accepted_Loans

,TableName,q1,Median,q3,lower_bound,upper_bound
0,annual_inc,40000.0,56712.0,80266.0,-20399.0,140665.0
1,delinq_2yrs,0.0,0.0,0.0,0.0,0.0
2,inq_last_6mths,0.0,1.0,2.0,-3.0,5.0
3,open_acc,6.0,9.0,12.0,-3.0,21.0
4,pub_rec,0.0,0.0,0.0,0.0,0.0
5,revol_util,23.9,47.8,71.9,-48.1,143.9
6,total_acc,13.0,20.0,29.0,-11.0,53.0
7,collections_12_mths_ex_med,0.0,0.0,0.0,0.0,0.0
8,acc_now_delinq,0.0,0.0,0.0,0.0,0.0


<h4> Lets check how many lower and upper outliers we have for each numeric column.</h4>

In [74]:
Outlier_Count = pd.DataFrame(columns=['TableName', 'Lower_Bound_Count', 'Upper_Bound_Count'])

for x in range(len(Box_Whiskers_Accepted_Loans)):
    col = Box_Whiskers_Accepted_Loans.iloc[x, 0]
    col_lower = Box_Whiskers_Accepted_Loans.iloc[x, 4] # lower bound value for this column
    col_upper = Box_Whiskers_Accepted_Loans.iloc[x, 5] # upper bound value for this column
    
    # Individual queries
    query_lower = f"SELECT COUNT(*) AS count FROM Accepted WHERE {col} < {col_lower}" # count the number of values below the lower bound for this column
    query_upper = f"SELECT COUNT(*) AS count FROM Accepted WHERE {col} > {col_upper}" # count the number of values above the upper bound for this column
    
    lower_count = pd.read_sql(query_lower, Connection).iloc[0]['count'] # amount of values below lower bound
    upper_count = pd.read_sql(query_upper, Connection).iloc[0]['count'] # amount of values above upper bound
    
    # Append to the outlier count dataframe
    Outlier_Count = pd.concat([
        Outlier_Count,
        pd.DataFrame([{
            'TableName': col,
            'Lower_Bound_Count': lower_count,
            'Upper_Bound_Count': upper_count
        }])
    ], ignore_index=True)


In [75]:
Outlier_Count

,TableName,Lower_Bound_Count,Upper_Bound_Count
0,annual_inc,0,1060
1,delinq_2yrs,0,2384
2,inq_last_6mths,0,732
3,open_acc,0,344
4,pub_rec,0,1233
5,revol_util,0,0
6,total_acc,0,319
7,collections_12_mths_ex_med,0,0
8,acc_now_delinq,0,4


<h4> Mean Imputation for numeric columns in  Accepted Loans</h4>

In [76]:
Accepted_Columns_Impute = ['annual_inc', 'delinq_2yrs','inq_last_6mths','open_acc','pub_rec', 'revol_util','total_acc','collections_12_mths_ex_med','acc_now_delinq']
for col in Accepted_Columns_Impute:
    Query = f" UPDATE Accepted SET {col} = ( SELECT AVG({col}) FROM Accepted WHERE {col} IS  NOT NULL ) WHERE {col} IS NULL" # replace every null value in each column with respecitve average value.
    Connection.execute(Query) # since we altering the table we have to execute the query , read_sql() is only for retriving data
    

In [77]:
# Lets Check to see if UPDATE was successful
Records_Check= pd.DataFrame({ 'TableName':[],'Null_Percentage':[]})
Total_Rows_Accepted = 20814 # number of records in the Accepted Loans

# We are running the same code again that we before to check for null vlaues
# In this case we are doing to see nulll values were replace with mean values
for col in modeling_columns:
    Query = f" Select {col} as TableName, count(*) as Null_Count from Accepted  WHERE {col}  IS NULL "
    Accepted_Loans_Result_Check = pd.read_sql(Query,Connection)
    Null_Count = Accepted_Loans_Result_Check['Null_Count'].iloc[0]
    Null_Percentage = (Null_Count/Total_Rows_Accepted)*100
    Records_Check = pd.concat([Records_Check, pd.DataFrame({ 'TableName': [col],'Null_Percentage': [Null_Percentage]})], ignore_index=True)

In [78]:
print(Records_Check)

                     TableName  Null_Percentage
0                    loan_amnt         0.000000
1                  funded_amnt         0.000000
2              funded_amnt_inv         0.000000
3                         term         0.000000
4                     int_rate         0.000000
5                  installment         0.000000
6                        grade         0.000000
7                    sub_grade         0.000000
8                   emp_length         1.695974
9               home_ownership         0.000000
10                  annual_inc         0.000000
11         verification_status         0.000000
12                     purpose         0.000000
13                  addr_state         0.000000
14                         dti         0.000000
15                 delinq_2yrs         0.000000
16              fico_range_low         0.000000
17             fico_range_high         0.000000
18              inq_last_6mths         0.000000
19                    open_acc         0

<h4> Based on the dataframe results, the columns with null values were successfully imputed.</h4>

<h4> Let's proceed and drop the columns that have 100% of data missing from them.</h4>

In [79]:
for col in Accepted_Columns_Drop:
    Query = f"  ALTER TABLE Accepted DROP COLUMN {col}" # we are going to alter the table by dropping the columns with 100% of data missing, after this query, we should have a lot less columns to work with
    Connection.execute(Query)

In [80]:
Query = f" PRAGMA table_info(Accepted) " # gives us metadata on table
Accepted_Loans_Query = pd.read_sql(Query,Connection)
Accepted_Columns = Accepted_Loans_Query['name'] # lets get the list of columns
print(Accepted_Columns)

0                      loan_amnt
1                    funded_amnt
2                funded_amnt_inv
3                           term
4                       int_rate
5                    installment
6                          grade
7                      sub_grade
8                     emp_length
9                 home_ownership
10                    annual_inc
11           verification_status
12                       issue_d
13                   loan_status
14                       purpose
15                    addr_state
16                           dti
17                   delinq_2yrs
18                fico_range_low
19               fico_range_high
20                inq_last_6mths
21                      open_acc
22                       pub_rec
23                     revol_bal
24                    revol_util
25                     total_acc
26    collections_12_mths_ex_med
27              application_type
28                acc_now_delinq
Name: name, dtype: object


<h4> As we can see, the columns were successfully dropped from the table.</h4>

In [81]:
# Awesome sauce, Let's check for duplicates
Cols_N_Table = ','.join(Accepted_Columns)
Query = f" SELECT {Cols_N_Table}, COUNT(*) as Count from Accepted GROUP BY {Cols_N_Table} HAVING COUNT(*) > 1" # let's check for duplicates in the accepted loans table based on all the columns
Result = pd.read_sql(Query,Connection)


In [82]:
Result # lets see if are are any duplicates

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,application_type,acc_now_delinq,Count


<h4> There are no duplicate values in the table, lets check for errors in categorical/non-numeric columns.</h4>

In [83]:
Query = " SELECT name from pragma_table_info('Accepted') WHERE type IN ('TEXT')" # let's check for Categorical columns
Cat_Cols = pd.read_sql(Query,Connection)
Cat_Cols

,name
0,term
1,grade
2,sub_grade
3,emp_length
4,home_ownership
5,verification_status
6,issue_d
7,loan_status
8,purpose
9,addr_state


In [84]:
Cat_Col_List = Cat_Cols['name']
Cat_Col_List

0                    term
1                   grade
2               sub_grade
3              emp_length
4          home_ownership
5     verification_status
6                 issue_d
7             loan_status
8                 purpose
9              addr_state
10       application_type
Name: name, dtype: object

In [85]:
for col in Cat_Col_List: # similar to value_counts() in pandas
    Query = f" SELECT {col}, COUNT(*) as freq from Accepted GROUP BY {col} ORDER BY freq DESC " # let's check for Categorical columns
    Result = pd.read_sql(Query,Connection)
    print( Result,"\n")


         term   freq
0   36 months  17433
1   60 months   3381 

  grade  freq
0     B  5824
1     C  4798
2     A  4429
3     D  3220
4     E  1655
5     F   579
6     G   309 

   sub_grade  freq
0         A5  1608
1         B5  1403
2         B4  1306
3         A4  1184
4         B3  1157
5         C1  1114
6         C2  1100
7         B2  1030
8         C3  1017
9         B1   928
10        A3   913
11        C4   821
12        D2   776
13        C5   746
14        D3   708
15        D1   647
16        D4   595
17        A2   535
18        D5   494
19        E1   436
20        E2   395
21        E3   322
22        E4   273
23        E5   229
24        A1   189
25        F1   161
26        F2   150
27        F3   108
28        F4    96
29        G4    71
30        G5    70
31        G1    65
32        F5    64
33        G2    57
34        G3    46 

   emp_length  freq
0   10+ years  4018
1    < 1 year  2954
2     2 years  2635
3     3 years  2263
4      1 year  2026
5     4 years  

<h4> It doesnt seem like any of the categorical/non-numeric columns have any irregularities, but based on addr_state and subgrade attributes, label encodning could exponentially increase dimenisonality during the modeling process. When we checked for nulls, emp_length had 2% of values missing , we will address this during the feature engineering for modeling.</h4>

<h2> Checking for Nulls, Duplicates, Negative values, Outliers and Categorical errors for Rejected Loans </h2>

<h4> Checking for Null values in each column</h4>

In [86]:
# Let's check for nulls in Rejected loans
# Lets get a list of column names
Query = f" PRAGMA table_info(Rejected) "
Rejected_Loans_Query = pd.read_sql(Query,Connection)
Rejected_Columns = list(Rejected_Loans_Query['name'])

In [87]:
Rejected_Columns

['Amount Requested',
 'Application Date',
 'Loan Title',
 'Risk_Score',
 'Debt-To-Income Ratio',
 'Zip Code',
 'State',
 'Employment Length',
 'Policy Code']

In [88]:
Rejected_Records = pd.DataFrame({ 'TableName':[],'Null_Percentage':[]}) # This will initailize an empty data frame  with coluns TableName and Null_Percentage
Total_Rows_Rejected = 200422 # number of records in the Rejected Loans

for col in Rejected_Columns:
    Rejected_Loans_Query = f" Select '{col}' as TableName, count(*) as Null_Count from Rejected  WHERE '{col}' IS NULL "  # Gather the null values from each column
    Rejected_Loans_Result = pd.read_sql(Rejected_Loans_Query,Connection) # store query  result in Rejected_Loans_ Result variable
    Null_Count = Rejected_Loans_Result['Null_Count'].iloc[0] # get the total number of null values for that column or attribute 
    Null_Percentage = (Null_Count/Total_Rows_Rejected)*100 # caluclate what percentage of the values for that column are null vlaues
    Rejected_Records = pd.concat([Rejected_Records, pd.DataFrame({ 'TableName': [col],'Null_Percentage': [Null_Percentage]})], ignore_index=True) # add column data to Rejected_Records dataframe, kinda like x = x+1

In [89]:
print(Rejected_Records )

              TableName  Null_Percentage
0      Amount Requested              0.0
1      Application Date              0.0
2            Loan Title              0.0
3            Risk_Score              0.0
4  Debt-To-Income Ratio              0.0
5              Zip Code              0.0
6                 State              0.0
7     Employment Length              0.0
8           Policy Code              0.0


<h4> Checking for Negative values in each numeric column</h4>

In [90]:
# Lets check for negatives values in the data 
# Lets check for negaive values, first we will gather all numeric columns
Query = " SELECT name from pragma_table_info('Rejected') WHERE type IN ('INTEGER', 'REAL', 'NUMERIC', 'DECIMAL', 'FLOAT', 'DOUBLE')" # This query will gather all numeric columns in the rjected loans table
Numeric_Cols_Rejected = pd.read_sql(Query,Connection)
Numeric_Cols_Rejected

,name
0,Amount Requested
1,Risk_Score
2,Policy Code


In [91]:
Numeric_Cols_Rejected = ["Amount Requested","Risk_Score","Policy Code"] # list of numeric columns
Cols_Neg_Count = pd.DataFrame({ 'Column':[],'Negative_Count':[]}) # empty dataframe


for col in Numeric_Cols_Rejected:
    Query = f" Select count(*) as Negative_Count from Rejected WHERE '{col}' < 0 " # gather all the negative values for the given column
    Neg_Count_Result = pd.read_sql(Query,Connection) 
    Cols_Neg_Count = pd.concat([ Cols_Neg_Count, pd.DataFrame({ 'Column':[col],'Negative_Count':[ Neg_Count_Result['Negative_Count'].iloc[0]]})], ignore_index=True) 

In [92]:
Cols_Neg_Count

,Column,Negative_Count
0,Amount Requested,0.0
1,Risk_Score,0.0
2,Policy Code,0.0


<h4> Results</h4>  
<h5>Based on these result, the numeric columns for the rejected loans table does not any negative values in the numeric columns.</h5>

I noticed that DTI has negative percentages and that's obviously not correct, so lets remove them.

In [93]:
DTI = '"Debt-To-Income Ratio"'
Query = f" Select count(*),{DTI} from Rejected where CAST({DTI} AS REAL) < 0 " # we cast the percentage values to real values in the DTI column and check if they are less than 0
Neg_DTI = pd.read_sql(Query,Connection)

In [38]:
Neg_DTI

,count(*),Debt-To-Income Ratio
0,9785,-1%


<h4> That's quite a bit of negative values, but this particular set of data is from the first 4 years so that makes sense.</h4>

In [94]:
Query = f" Delete from Rejected where CAST({DTI} AS REAL) < 0 " # cehck for the negative values in the DTI column and delete them from the Rejected loans table.
Connection.execute(Query)

In [95]:
Query = f" Select count(*) as Negative_Values_Count,{DTI} from Rejected where CAST({DTI} AS REAL) < 0 " # Let's check if negative percentages are gone.
DTI_Neg_Check = pd.read_sql(Query,Connection)

In [96]:
DTI_Neg_Check

,Negative_Values_Count,Debt-To-Income Ratio
0,0,None


<h3> Should we check for duplicates? </h3>

We checked for duplicates with Accepted loans but after further inspection. someone having duplicates in the same day makes sense for several reasons: 

-  Multiple applications put in by the same person, 
-  Lending club could log multiple attempts from different channles ( e.g wesite , mobile app, etc).
-  No unique identifiers so their is no way to tell if the application is by the same person or different people.

<h4> Check for Outliers in Numeric columns for Rejected Loans</h4>

In [97]:
Rejected_Loans.head()

Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
f64,str,str,f64,str,str,str,str,f64
1000.0,"""2007-05-26""","""Wedding Covered but No Honeymo…",693.0,"""10%""","""481xx""","""NM""","""4 years""",0.0
1000.0,"""2007-05-26""","""Consolidating Debt""",703.0,"""10%""","""010xx""","""MA""","""< 1 year""",0.0
11000.0,"""2007-05-27""","""Want to consolidate my debt""",715.0,"""10%""","""212xx""","""MD""","""1 year""",0.0
6000.0,"""2007-05-27""","""waksman""",698.0,"""38.64%""","""017xx""","""MA""","""< 1 year""",0.0
1500.0,"""2007-05-27""","""mdrigo""",509.0,"""9.43%""","""209xx""","""MD""","""< 1 year""",0.0


In [98]:
Box_Whiskers_Rejected_Loans = pd.DataFrame(columns=['TableName', 'q1', 'Median', 'q3', 'lower_bound', 'upper_bound'])

num_col = ["Amount Requested","Risk_Score","Policy Code"]

for col in num_col:
    count_query = f"SELECT COUNT(*) as total from Rejected"
    total_result = pd.read_sql(count_query, Connection)
    total = total_result['total'].iloc[0]
    

    # Calculate the offsets for Q1, Median, Q3
    q1_offset = int(total * 0.25)
    median_offset = int(total * 0.5)
    q3_offset = int(total * 0.75)

    # Now use those offsets in a separate query
    Query = f"""
    WITH ordered AS (
      SELECT "{col}"
      FROM Rejected
      WHERE "{col}" IS NOT NULL
      ORDER BY "{col}"
    )
    SELECT 
      (SELECT "{col}" FROM ordered LIMIT 1 OFFSET {q1_offset}) AS q1,
      (SELECT "{col}" FROM ordered LIMIT 1 OFFSET {median_offset}) AS median,
      (SELECT "{col}" FROM ordered LIMIT 1 OFFSET {q3_offset}) AS q3
    """
    
    Box_Result = pd.read_sql(Query, Connection)
    q1 = Box_Result['q1'].iloc[0]
    median = Box_Result['median'].iloc[0]
    q3 = Box_Result['q3'].iloc[0]
    lower_bound = q1 - 1.5 * (q3 - q1)
    upper_bound = q3 + 1.5 * (q3 - q1)

    Box_Whiskers_Rejected_Loans = pd.concat([Box_Whiskers_Rejected_Loans,
    pd.DataFrame({'TableName': [col],'q1': [q1],'Median': [median],'q3': [q3],'lower_bound': [lower_bound],'upper_bound': [upper_bound]})], ignore_index=True)


/tmp/ipykernel_35/4152677464.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Box_Whiskers_Rejected_Loans = pd.concat([Box_Whiskers_Rejected_Loans,


In [99]:
Box_Whiskers_Rejected_Loans

,TableName,q1,Median,q3,lower_bound,upper_bound
0,Amount Requested,4000.0,8000.0,15000.0,-12500.0,31500.0
1,Risk_Score,545.0,642.0,693.0,323.0,915.0
2,Policy Code,0.0,0.0,0.0,0.0,0.0


In [100]:
Outlier_Count = pd.DataFrame(columns=['TableName', 'Lower_Bound_Count', 'Upper_Bound_Count'])

for x in range(len(Box_Whiskers_Rejected_Loans)):
    col = Box_Whiskers_Rejected_Loans.iloc[x, 0]
    col_lower = Box_Whiskers_Rejected_Loans.iloc[x, 4]
    col_upper = Box_Whiskers_Rejected_Loans.iloc[x, 5]
    quoted_col = f'"{col}"'
    # Individual queries
    query_lower = f""" SELECT COUNT(*) AS count FROM Rejected WHERE "{col}" < {col_lower} """
    query_upper = f"""SELECT COUNT(*) AS count FROM Rejected WHERE "{col}" > {col_upper} """
    
    lower_count = pd.read_sql(query_lower, Connection).iloc[0]['count']
    upper_count = pd.read_sql(query_upper, Connection).iloc[0]['count']
    
    # Append to the outlier count dataframe
    Outlier_Count = pd.concat([
        Outlier_Count,
        pd.DataFrame([{
            'TableName': col,
            'Lower_Bound_Count': lower_count,
            'Upper_Bound_Count': upper_count
        }])
    ], ignore_index=True)


In [101]:
Outlier_Count

,TableName,Lower_Bound_Count,Upper_Bound_Count
0,Amount Requested,0,52
1,Risk_Score,13407,0
2,Policy Code,0,0


<h4> Categorical Errors check for Rejected Loans</h4>

In [102]:
Query = f" PRAGMA table_info(Rejected) "
Rejected_Loans_Query = pd.read_sql(Query,Connection)
Rejected_Loans_Query

,cid,name,type,notnull,dflt_value,pk
0,0,Amount Requested,REAL,0,None,0
1,1,Application Date,TEXT,0,None,0
2,2,Loan Title,TEXT,0,None,0
3,3,Risk_Score,REAL,0,None,0
4,4,Debt-To-Income Ratio,TEXT,0,None,0
5,5,Zip Code,TEXT,0,None,0
6,6,State,TEXT,0,None,0
7,7,Employment Length,TEXT,0,None,0
8,8,Policy Code,REAL,0,None,0


In [103]:
Query = " SELECT name from pragma_table_info('Rejected') WHERE type IN ('TEXT')" # let's check for Categorical columns
Cat_Cols_Rejected = pd.read_sql(Query,Connection)
Cat_Cols_Rejected

,name
0,Application Date
1,Loan Title
2,Debt-To-Income Ratio
3,Zip Code
4,State
5,Employment Length


In [105]:
Cat_Cols_Rejected_List = list( Cat_Cols_Rejected['name'])

In [106]:
for col in Cat_Cols_Rejected_List:
    Query = f""" SELECT "{col}", COUNT(*) as freq from Rejected GROUP BY "{col}" ORDER BY freq DESC """ # let's check for Categorical columns
    Result = pd.read_sql(Query,Connection)
    print( Result,"\n")

     Application Date  freq
0          2010-05-11   707
1          2010-11-16   577
2          2010-09-28   571
3          2008-04-07   562
4          2010-10-19   560
...               ...   ...
1311       2007-08-19     2
1312       2007-07-28     2
1313       2007-06-16     2
1314       2007-05-26     2
1315       2007-06-07     1

[1316 rows x 2 columns] 

                                              Loan Title   freq
0                                     debt_consolidation  39839
1                                                         15219
2                                                  other  14456
3                                            credit_card   7951
4                                         major_purchase   7899
...                                                  ...    ...
45916    Borrower added on 08/25/10 > These are the e...      1
45917    Borrower added on 08/09/10 > Assets are requ...      1
45918    Borrower added on 06/11/10 > To pay off cred...     

In [108]:
# Noticed some weird values for DTI again, there's no wway 99999% DTI can be an actual value nor can 199998%, a value is 100% is already risky , anything higher than higher than is probably errorneous 
DTI = '"Debt-To-Income Ratio"'
Query = f" Select count(*) as count from Rejected where CAST({DTI} AS REAL) > 100" # we cast the percentage values to real values in the DTI column and check if they are less than 0
DTI_Error = pd.read_sql(Query,Connection)

In [109]:
DTI_Error

,count
0,7731


In [110]:
Query = f" Delete from Rejected where CAST({DTI} AS REAL) > 100 " # cehck for the negative values in the DTI column and delete them from the Rejected loans table.
Connection.execute(Query)

In [113]:
Query = f" Select count(*) as count from Rejected where CAST({DTI} AS REAL) > 100" # we cast the percentage values to real values in the DTI column and check if they are less than 0
DTI_Error = pd.read_sql(Query,Connection)

In [114]:
DTI_Error

,count
0,0


<h2> Lets save our cleaned datasets for further processing via pandas/polars</h2>

<h3> Saving Accepted Loans</h3>

In [115]:
Clean_Accepted_Loans = pl.read_database("SELECT * FROM Accepted", connection=Connection)

In [116]:
Clean_Accepted_Loans.head()

loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,application_type,acc_now_delinq
f64,f64,f64,str,f64,f64,str,str,str,str,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
20000.0,20000.0,16358.045925,""" 60 months""",9.99,424.85,"""B""","""B4""","""7 years""","""MORTGAGE""",45000.0,"""Verified""","""Dec-2010""","""Fully Paid""","""debt_consolidation""","""CA""",13.36,0.0,760.0,764.0,2.0,14.0,0.0,12761.0,39.5,33.0,0.0,"""Individual""",0.0
7475.0,7475.0,7475.0,""" 60 months""",13.72,172.85,"""C""","""C5""","""< 1 year""","""OWN""",145000.0,"""Not Verified""","""Dec-2010""","""Fully Paid""","""debt_consolidation""","""GA""",17.39,0.0,730.0,734.0,1.0,11.0,0.0,42581.0,44.8,30.0,0.0,"""Individual""",0.0
5575.0,5575.0,5575.0,""" 36 months""",15.2,193.81,"""D""","""D4""","""< 1 year""","""MORTGAGE""",120000.0,"""Not Verified""","""Dec-2010""","""Fully Paid""","""small_business""","""MD""",16.4,1.0,685.0,689.0,1.0,14.0,0.0,10611.0,48.0,48.0,0.0,"""Individual""",0.0
2150.0,2150.0,2150.0,""" 60 months""",14.83,50.96,"""D""","""D3""","""< 1 year""","""RENT""",120000.0,"""Not Verified""","""Dec-2010""","""Fully Paid""","""debt_consolidation""","""NY""",6.16,0.0,720.0,724.0,0.0,5.0,0.0,20020.0,57.9,6.0,0.0,"""Individual""",0.0
7050.0,7050.0,7050.0,""" 60 months""",12.98,160.34,"""C""","""C3""","""< 1 year""","""MORTGAGE""",140000.0,"""Not Verified""","""Dec-2010""","""Fully Paid""","""moving""","""CA""",4.26,0.0,810.0,814.0,1.0,6.0,0.0,78.0,0.2,17.0,0.0,"""Individual""",0.0


In [119]:
Clean_Accepted_Loans.write_csv("cleaned_accepted_loans_2007-2010.csv")

<h3> Saving Rejected Loans</h3>

In [120]:
Clean_Rejected_Loans = pl.read_database("SELECT * FROM Rejected", connection=Connection)

In [121]:
Clean_Rejected_Loans.head()

Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
f64,str,str,f64,str,str,str,str,f64
1000.0,"""2007-05-26""","""Wedding Covered but No Honeymo…",693.0,"""10%""","""481xx""","""NM""","""4 years""",0.0
1000.0,"""2007-05-26""","""Consolidating Debt""",703.0,"""10%""","""010xx""","""MA""","""< 1 year""",0.0
11000.0,"""2007-05-27""","""Want to consolidate my debt""",715.0,"""10%""","""212xx""","""MD""","""1 year""",0.0
6000.0,"""2007-05-27""","""waksman""",698.0,"""38.64%""","""017xx""","""MA""","""< 1 year""",0.0
1500.0,"""2007-05-27""","""mdrigo""",509.0,"""9.43%""","""209xx""","""MD""","""< 1 year""",0.0


In [122]:
Clean_Rejected_Loans.write_csv("cleaned_rejected_loans_2007-2010.csv")